In [2]:
# Load packages
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import os
import pickle
import time
import scipy as scp
import scipy.stats as scps
from scipy.optimize import differential_evolution
from scipy.optimize import minimize
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Load my own functions
import dnnregressor_train_eval_keras as dnnk
import make_data_wfpt as mdw
from kde_training_utilities import kde_load_data
import ddm_data_simulation as ddm_sim
import boundary_functions as bf

In [3]:
# Handle some cuda business

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8468172253313650259
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 1663136966598024423
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 12048649421
locality {
  bus_id: 2
  numa_node: 1
  links {
  }
}
incarnation: 7634277065132063798
physical_device_desc: "device: 0, name: GeForce GTX TITAN X, pci bus id: 0000:82:00.0, compute capability: 5.2"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 9028000898248782928
physical_device_desc: "device: XLA_GPU device"
]


In [6]:
# Load Model
model_path = '/media/data_cifs/afengler/data/kde/full_ddm/keras_models/dnnregressor_full_ddm_06_28_19_00_48_00/model_0' 
ckpt_path = '/media/data_cifs/afengler/data/kde/full_ddm/keras_models/dnnregressor_full_ddm_06_28_19_00_48_00/ckpt_0_10'

model = keras.models.load_model(model_path)
model.load_weights(ckpt_path)

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 7)                 0         
_________________________________________________________________
dense_7 (Dense)              (None, 20)                160       
_________________________________________________________________
dense_8 (Dense)              (None, 40)                840       
_________________________________________________________________
dense_9 (Dense)              (None, 60)                2460      
_________________________________________________________________
dense_10 (Dense)             (None, 80)                4880      
_________________________________________________________________
dense_11 (Dense)             (None, 100)               8100      
_________________________________________________________________
dense_12 (Dense)             (None, 120)               12120     
__________

In [4]:
# Initializations -----
n_runs = 100
n_samples = 2500
feature_file_path = '/media/data_cifs/afengler/data/kde/ornstein_uhlenbeck/train_test_data/test_features.pickle'
mle_out_path = '/media/data_cifs/afengler/data/kde/ornstein_uhlenbeck/mle_runs'

# NOTE PARAMETERS: WEIBULL: [v, a, w, node, shape, scale]
param_bounds = [(-2, 2), (0.5, 2), (0.3, 0.7), (-1.0, 1.0)]


# my_optim_columns = ['v_sim', 'a_sim', 'w_sim', 'node_sim', 'theta_sim',
#                     'v_mle', 'a_mle', 'w_mle', 'node_mle', 'theta_mle', 'n_samples']

# Get parameter names in correct ordering:
dat = pickle.load(open(feature_file_path, 
                       'rb'))

parameter_names = list(dat.keys())[:-2] # :-1 to get rid of 'rt' and 'choice' here

# Make columns for optimizer result table
p_sim = []
p_mle = []

for parameter_name in parameter_names:
    p_sim.append(parameter_name + '_sim')
    p_mle.append(parameter_name + '_mle')
    
my_optim_columns = p_sim + p_mle + ['n_samples']

# Initialize the data frame in which to store optimizer results
optim_results = pd.DataFrame(np.zeros((n_runs, len(my_optim_columns))), columns = my_optim_columns)
optim_results.iloc[:, 2 * len(parameter_names)] = n_samples

# define boundary
boundary = bf.constant
boundary_multiplicative = True

# Define the likelihood function
def log_p(params = [0, 1, 0.9], model = [], data = [], parameter_names = []):
    # Make feature array
    feature_array = np.zeros((data[0].shape[0], len(parameter_names) + 2))
    
    # Store parameters
    cnt = 0
    for i in range(0, len(parameter_names), 1):
        feature_array[:, i] = params[i]
        cnt += 1
    
    # Store rts and choices
    feature_array[:, cnt] = data[0].ravel() # rts
    feature_array[:, cnt + 1] = data[1].ravel() # choices
    
    # Get model predictions
    prediction = model.predict(feature_array)
    
    # Some post-processing of predictions
    prediction[prediction < 1e-29] = 1e-29
    
    return(- np.sum(np.log(prediction)))  

def make_params(param_bounds = []):
    params = np.zeros(len(param_bounds))
    
    for i in range(len(params)):
        params[i] = np.random.uniform(low = param_bounds[i][0], high = param_bounds[i][1])
        
    return params
# ---------------------

In [ ]:
my_optim_columns

In [ ]:
# Main loop ----------- TD: Parallelize
for i in range(0, n_runs, 1): 
    
    # Get start time
    start_time = time.time()
    
    tmp_params = make_params(param_bounds = param_bounds)
    
    # Store in output file
    optim_results.iloc[i, :len(parameter_names)] = tmp_params
    
    # Print some info on run
    print('Parameters for run ' + str(i) + ': ')
    print(tmp_params)
    
    # Define boundary params
    # Linear Collapse
#     boundary_params = {'node': tmp_params[3],
#                        'theta': tmp_params[4]}
    
    # Constant
    boundary_params = {}
    
    # Run model simulations
    ddm_dat_tmp = ddm_sim.ddm_flexbound_simulate(v = tmp_params[0],
                                                 a = tmp_params[1],
                                                 w = tmp_params[2],
                                                 s = 1,
                                                 delta_t = 0.001,
                                                 max_t = 20,
                                                 n_samples = n_samples,
                                                 boundary_fun = boundary, # function of t (and potentially other parameters) that takes in (t, *args)
                                                 boundary_multiplicative = boundary_multiplicative, # CAREFUL: CHECK IF BOUND
                                                 boundary_params = boundary_params)
        
    # Print some info on run
    print('Mean rt for current run: ')
    print(np.mean(ddm_dat_tmp[0]))
    
    # Run optimizer
    out = differential_evolution(log_p, 
                                 bounds = param_bounds, 
                                 args = (model, ddm_dat_tmp, parameter_names), 
                                 popsize = 30,
                                 disp = True)
    
    # Print some info
    print('Solution vector of current run: ')
    print(out.x)
    
    print('The run took: ')
    elapsed_time = time.time() - start_time
    print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))
    
    # Store result in output file
    optim_results.iloc[i, len(parameter_names):(2*len(parameter_names))] = out.x
# -----------------------

# Save optimization results to file
optim_results.to_csv(mle_out_path + '/mle_results_1.csv')

Parameters for run 0: 
[0.87209855 1.43430007 0.522945   0.81011296]
0  datapoints sampled
1000  datapoints sampled
2000  datapoints sampled
Mean rt for current run: 
1.3427599999999527
differential_evolution step 1: f(x)= 3482.38
differential_evolution step 2: f(x)= 3342.78
differential_evolution step 3: f(x)= 3342.78
differential_evolution step 4: f(x)= 3342.78
differential_evolution step 5: f(x)= 3342.78
differential_evolution step 6: f(x)= 3342.78
differential_evolution step 7: f(x)= 3342.78
differential_evolution step 8: f(x)= 3342.78
differential_evolution step 9: f(x)= 3342.78
differential_evolution step 10: f(x)= 3342.78
differential_evolution step 11: f(x)= 3342.78
differential_evolution step 12: f(x)= 3342.78
differential_evolution step 13: f(x)= 3342.78
differential_evolution step 14: f(x)= 3342.49
differential_evolution step 15: f(x)= 3342.49
differential_evolution step 16: f(x)= 3342.49
differential_evolution step 17: f(x)= 3342.49
differential_evolution step 18: f(x)= 334

1000  datapoints sampled
2000  datapoints sampled
Mean rt for current run: 
3.852743599999936
differential_evolution step 1: f(x)= 7138.57
differential_evolution step 2: f(x)= 7062.61
differential_evolution step 3: f(x)= 7062.61
differential_evolution step 4: f(x)= 7062.61
differential_evolution step 5: f(x)= 7062.02
differential_evolution step 6: f(x)= 7051.34
differential_evolution step 7: f(x)= 7048.59
differential_evolution step 8: f(x)= 7048.59
differential_evolution step 9: f(x)= 7037.77
differential_evolution step 10: f(x)= 7036.92
differential_evolution step 11: f(x)= 7028.68
differential_evolution step 12: f(x)= 7018.02
differential_evolution step 13: f(x)= 7015.28
Solution vector of current run: 
[-0.28778626  1.9884777   0.57618779 -0.36736004]
The run took: 
00:05:49
Parameters for run 6: 
[-1.71126567  1.68334506  0.59534796 -0.36897827]
0  datapoints sampled
1000  datapoints sampled
2000  datapoints sampled
Mean rt for current run: 
1.1470835999999687
differential_evoluti

1000  datapoints sampled
2000  datapoints sampled
Mean rt for current run: 
0.2504572
differential_evolution step 1: f(x)= -181.051
differential_evolution step 2: f(x)= -270.642
differential_evolution step 3: f(x)= -297.194
differential_evolution step 4: f(x)= -329.912
differential_evolution step 5: f(x)= -329.912
differential_evolution step 6: f(x)= -329.912
differential_evolution step 7: f(x)= -347.237
differential_evolution step 8: f(x)= -351.062
differential_evolution step 9: f(x)= -352.434
differential_evolution step 10: f(x)= -360.194
differential_evolution step 11: f(x)= -360.194
differential_evolution step 12: f(x)= -360.682
differential_evolution step 13: f(x)= -360.682
differential_evolution step 14: f(x)= -380.417
differential_evolution step 15: f(x)= -380.417
differential_evolution step 16: f(x)= -396.435
differential_evolution step 17: f(x)= -396.435
differential_evolution step 18: f(x)= -396.435
differential_evolution step 19: f(x)= -397.443
differential_evolution step 20

differential_evolution step 6: f(x)= 1649.38
differential_evolution step 7: f(x)= 1649.38
differential_evolution step 8: f(x)= 1630.62
differential_evolution step 9: f(x)= 1583.25
differential_evolution step 10: f(x)= 1583.25
differential_evolution step 11: f(x)= 1583.25
differential_evolution step 12: f(x)= 1583.25
differential_evolution step 13: f(x)= 1583.25
differential_evolution step 14: f(x)= 1583.25
differential_evolution step 15: f(x)= 1583.25
differential_evolution step 16: f(x)= 1583.25
differential_evolution step 17: f(x)= 1583.25
differential_evolution step 18: f(x)= 1583.25
differential_evolution step 19: f(x)= 1583.25
differential_evolution step 20: f(x)= 1583.25
differential_evolution step 21: f(x)= 1583.25
differential_evolution step 22: f(x)= 1583.25
differential_evolution step 23: f(x)= 1583.25
differential_evolution step 24: f(x)= 1583.25
differential_evolution step 25: f(x)= 1583.25
differential_evolution step 26: f(x)= 1583.25
differential_evolution step 27: f(x)= 

1000  datapoints sampled
2000  datapoints sampled
Mean rt for current run: 
0.7885755999999886
differential_evolution step 1: f(x)= 1189.74
differential_evolution step 2: f(x)= 980.833
differential_evolution step 3: f(x)= 980.833
differential_evolution step 4: f(x)= 980.833
differential_evolution step 5: f(x)= 980.833
differential_evolution step 6: f(x)= 980.833
differential_evolution step 7: f(x)= 980.833
differential_evolution step 8: f(x)= 980.833
differential_evolution step 9: f(x)= 980.833
differential_evolution step 10: f(x)= 980.833
differential_evolution step 11: f(x)= 980.833
differential_evolution step 12: f(x)= 980.833
differential_evolution step 13: f(x)= 980.833
differential_evolution step 14: f(x)= 980.833
differential_evolution step 15: f(x)= 980.833
differential_evolution step 16: f(x)= 980.833
differential_evolution step 17: f(x)= 977.048
differential_evolution step 18: f(x)= 975.996
differential_evolution step 19: f(x)= 975.883
differential_evolution step 20: f(x)= 97

1000  datapoints sampled
2000  datapoints sampled
Mean rt for current run: 
0.8291143999999853
differential_evolution step 1: f(x)= 1484.27
differential_evolution step 2: f(x)= 1480.06
differential_evolution step 3: f(x)= 1470.76
differential_evolution step 4: f(x)= 1426.39
differential_evolution step 5: f(x)= 1415.15
differential_evolution step 6: f(x)= 1380.36
differential_evolution step 7: f(x)= 1380.36
differential_evolution step 8: f(x)= 1380.36
differential_evolution step 9: f(x)= 1362.19
differential_evolution step 10: f(x)= 1353.94
differential_evolution step 11: f(x)= 1314.14
differential_evolution step 12: f(x)= 1314.14
differential_evolution step 13: f(x)= 1314.14
differential_evolution step 14: f(x)= 1314.14
differential_evolution step 15: f(x)= 1314.14
differential_evolution step 16: f(x)= 1314.14
differential_evolution step 17: f(x)= 1314.14
differential_evolution step 18: f(x)= 1314.14
differential_evolution step 19: f(x)= 1296.69
differential_evolution step 20: f(x)= 12

In [ ]:
# Read in results
optim_results = pd.read_csv(mle_out_path + '/mle_results.csv')

In [ ]:
plt.scatter(optim_results['v_sim'], optim_results['v_mle'], c = optim_results['theta_mle'])

In [ ]:
# Regression for v
reg = LinearRegression().fit(np.expand_dims(optim_results['v_mle'], 1), np.expand_dims(optim_results['v_sim'], 1))
reg.score(np.expand_dims(optim_results['v_mle'], 1), np.expand_dims(optim_results['v_sim'], 1))

In [ ]:
plt.scatter(optim_results['a_sim'], optim_results['a_mle'], c = optim_results['theta_mle'])

In [ ]:
# Regression for a
reg = LinearRegression().fit(np.expand_dims(optim_results['a_mle'], 1), np.expand_dims(optim_results['a_sim'], 1))
reg.score(np.expand_dims(optim_results['a_mle'], 1), np.expand_dims(optim_results['a_sim'], 1))

In [ ]:
plt.scatter(optim_results['w_sim'], optim_results['w_mle'])

In [ ]:
# Regression for w
reg = LinearRegression().fit(np.expand_dims(optim_results['w_mle'], 1), np.expand_dims(optim_results['w_sim'], 1))
reg.score(np.expand_dims(optim_results['w_mle'], 1), np.expand_dims(optim_results['w_sim'], 1))

In [ ]:
plt.scatter(optim_results['theta_sim'], optim_results['theta_mle'])

In [ ]:
# Regression for c1
reg = LinearRegression().fit(np.expand_dims(optim_results['theta_mle'], 1), np.expand_dims(optim_results['theta_sim'], 1))
reg.score(np.expand_dims(optim_results['theta_mle'], 1), np.expand_dims(optim_results['theta_sim'], 1))

In [ ]:
plt.scatter(optim_results['c2_sim'], optim_results['c2_mle'], c = optim_results['a_mle'])

In [ ]:
# Regression for w
reg = LinearRegression().fit(np.expand_dims(optim_results['c2_mle'], 1), np.expand_dims(optim_results['c2_sim'], 1))
reg.score(np.expand_dims(optim_results['c2_mle'], 1), np.expand_dims(optim_results['c2_sim'], 1))